In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
!ls ../../modin/mix_fire.csv

### Note: This dataset is about is 700MB compressed and 3.2GB uncompressed with 37M records
- Please contact me either @prodramp twitter or use YouTube comment from this turorial and I will send you the google drive link for get this dataset
- I am in process to create full global hotspot dataset with over 250M records and store at Kaggle. Please kaggle for that dataset, I will try to update here too if I could make it. 
- Thanks.


In [ ]:
source = "../../modin/mix_fire.csv"

In [ ]:
df = pd.read_csv(source, low_memory=False)

In [ ]:
df

In [ ]:
df['acq_date'] = pd.to_datetime(df['acq_date'])

In [ ]:
daily_df = df.groupby(
            ['latitude', 'longitude', 'acq_date', 'satellite', 'instrument']).confidence.max().reset_index()

In [ ]:
daily_df

In [ ]:
daily_df['year'] = daily_df.acq_date.dt.year

In [ ]:
daily_df['month'] = daily_df.acq_date.dt.month

In [ ]:
PRECISION = 1

In [ ]:
# Selecting only 4 columns 
df = daily_df[['latitude', 'longitude', 'acq_date', 'confidence']]

# setting up the each data point of about 10sq KM by using single decimal precision
df.latitude = df.latitude.round(PRECISION)
df.longitude = df.longitude.round(PRECISION)

# Getting year and month from the date and creating new columns
df['year'] = df.acq_date.dt.year
df['month'] = df.acq_date.dt.month

# grouping data on 4 selected fields
total_fires = df.groupby(['latitude', 'longitude', 'year', 'month']).size().reset_index()
# reseting the last size column name to fire_count
total_fires.columns = ['latitude', 'longitude', 'year', 'month', 'fire_count']

# creating yearly and total fire count with groupBy
yearly_fires = total_fires.groupby(['longitude', 'latitude', 'year', ]).count().reset_index()
global_fires = total_fires.groupby(['latitude', 'longitude']).sum().reset_index()

In [ ]:
yearly_fires

In [ ]:
global_fires

### each 0 decimal is about 110Km and 1 decimal place in longitude and latitude is about 11KM distance
- http://wiki.gis.com/wiki/index.php/Decimal_degrees

In [ ]:
total_fires_100 = total_fires
total_fires_100.latitude = total_fires_100.latitude.round(0)
total_fires_100.longitude = total_fires_100.longitude.round(0)
total_fires_100

In [ ]:
total_fires_100 = total_fires_100.groupby(['latitude', 'longitude'])['fire_count'].sum().reset_index()
total_fires_100

### Please visit to https://mapbox.com if following this MAPBOX_TOKEN is invalid

In [ ]:
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
MAPBOX_TOKEN = 'pk.eyJ1IjoiZm9kZ2Fib3JtYXRoIiwiYSI6ImNrZmY3Nzc2bjBiemkyeG8zdGNzcXgzMGIifQ.J0dZhMiuZTPVexL8nrpS6Q'
px.set_mapbox_access_token(MAPBOX_TOKEN)

In [ ]:
fig = px.scatter_geo(total_fires_100, 
                     lon = total_fires_100['longitude'],
                     lat = total_fires_100['latitude'], 
                     color="fire_count",
                     hover_name="fire_count", size="fire_count",
                     projection="natural earth")
fig.show()

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=global_fires, lat='latitude', lon='longitude',
    nx_hexagon=150, opacity=0.9, labels={"color": "Hotspot records"},
    color='fire_count', agg_func=np.sum, color_continuous_scale="Reds"
)
fig.show()

In [ ]:
global_fires.columns

In [ ]:
fig = px.density_mapbox(global_fires, lat='latitude', lon='longitude', z='fire_count', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()